______

# Graded Assignment: Classification in Machine Learning

Kuis ini merupakan bagian dari proses penilaian Algoritma Training. Selamat atas selesainya materi Classification dalam Machine Learning. Kami akan melakukan penilaian berupa kuis untuk menguji praktek teknik pembuatan model regresi dan klasifikasi yang sudah Anda pelajari.

Pada quiz berikut, Anda akan diminta membuat model klasifikasi sebagai penilaian pemahaman mengenai materi yang sudah dipelajari sebelumnya.

## Credit Risk Analysis

Ketika bank menerima permohonan pinjaman, berdasarkan profil pemohon maka bank harus membuat keputusan apakah akan melanjutkan persetujuan pinjaman atau tidak. Dua jenis resiko terkait keputusan bank: 

- Jika pemohon memiliki risiko kredit yang baik, yaitu mungkin untuk membayar kembali pinjaman, maka tidak menyetujui pinjaman kepada orang tersebut mengakibatkan kerugian bisnis bagi bank
- Jika pemohon memiliki risiko kredit yang buruk, yaitu tidak mungkin untuk membayar kembali pinjaman, maka menyetujui pinjaman kepada orang tersebut mengakibatkan kerugian finansial bagi bank.

### Import data

Kali ini kita akan menggunakan data `credit_germany.csv` yang tersimpan pada folder `data_input`. Data ini merupakan data real dari suatu bank yang ada di German.

In [26]:
import pandas as pd
import statsmodels.api as sm

##  code here
credit = pd.read_csv('data_input/credit_germany.csv')
credit.head()

,Creditability,Account_Balance,Duration_of_Credit_monthly,Payment_Status_of_Previous_Credit,Credit_Amount,Length_of_current_employment,Guarantors,Duration_in_Current_address,Age_years,Concurrent_Credits,No_of_Credits_at_this_Bank
0,1,No Account,18,No_Problems,1049,2,No,4,21,Other_Bank,1
1,1,No Account,9,No_Problems,2799,3,No,2,36,Other_Bank,2
2,1,Have Account,12,Paid_Up,841,4,No,4,23,Other_Bank,1
3,1,No Account,12,No_Problems,2122,3,No,2,39,Other_Bank,2
4,1,No Account,12,No_Problems,2171,3,No,4,38,No_credit,2


In [27]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Creditability                      1000 non-null   int64 
 1   Account_Balance                    1000 non-null   object
 2   Duration_of_Credit_monthly         1000 non-null   int64 
 3   Payment_Status_of_Previous_Credit  1000 non-null   object
 4   Credit_Amount                      1000 non-null   int64 
 5   Length_of_current_employment       1000 non-null   int64 
 6   Guarantors                         1000 non-null   object
 7   Duration_in_Current_address        1000 non-null   int64 
 8   Age_years                          1000 non-null   int64 
 9   Concurrent_Credits                 1000 non-null   object
 10  No_of_Credits_at_this_Bank         1000 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 93.8+ KB


**Data Description:**

- `Creditability` : Apakah peminjam memiliki risiko kredit yang buruk (0) atau kredit baik (1)
- `Account_Balance` : Kategori apakah peminjam memiliki akun bank
    - No account = tidak memiliki akun bank
    - Have account = memiliki akun dan balance pada bank
- `Duration_of_Credit_monthly` : Durasi pinjaman dalam bulan
- `Payment_Status_of_Previous_Credit` : Histori pembayaran dari kredit yang pernah diajukan sebelumnya
    - No_loan_history = Tidak memiliki riwayat pinjaman
    - Some_Problems = Terlambat bayar dan belum diselesaikan pembayarannya 
    - Paid_Up = Pernah terlambat bayar dan sudah diselesaikan pembayarannya 
    - No_Problems = Tidak ada masalah
- `Credit_Amount` : Jumlah nominal pinjaman yang diajukan
- `Length_of_current_employment` : Lama bekerja di tempat kerja sekarang dalam tahun
- `Guarantors` : Apakah terdapat penjamin pinjaman atau tidak
    - No = Tidak ada penjamin
    - Yes =  Ada penjamin
- `Duration_in_Current_address` : Durasi lama tinggal di tempat tinggal sekarang dalam tahun
- `Age_years` : Usia peminjam
- `Concurrent_Credits`: Tipe kredit yang sedang berjalan bersamaan
    - No_credit = Tidak ada kredit yang jalan bersamaan 
    - Dept_Stores = Credit terhadap department store 
    - Other_Bank = Credit terhadap bank lain
- `No_of_Credits_at_this_Bank`: Jumlah kredit yang sedang berjalan pada bank ini

Sebelum masuk pada tahap pembuatan model, kita akan melakukan EDA untuk mengetahui variabel prediktor yang perlu dimasukkan dalam model dan yang tidak.

### Wrangling Data

#### Mengubah tipe data

Sebelum melakukan perubahan tipe data, silakan cek terlebih dahulu jenis tipe datanya dengan menggunakan method `dtypes`/`info()`

In [28]:
# # list berisi nama kolom yang ingin diubah dalam format sama
cat = ['Account_Balance', 'Payment_Status_of_Previous_Credit', 'Guarantors', 'Concurrent_Credits']

# # Mengubah tipe data beberapa kolom
credit[cat] = credit[cat].astype('category')

# # cek kembali tipe data
credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   Creditability                      1000 non-null   int64   
 1   Account_Balance                    1000 non-null   category
 2   Duration_of_Credit_monthly         1000 non-null   int64   
 3   Payment_Status_of_Previous_Credit  1000 non-null   category
 4   Credit_Amount                      1000 non-null   int64   
 5   Length_of_current_employment       1000 non-null   int64   
 6   Guarantors                         1000 non-null   category
 7   Duration_in_Current_address        1000 non-null   int64   
 8   Age_years                          1000 non-null   int64   
 9   Concurrent_Credits                 1000 non-null   category
 10  No_of_Credits_at_this_Bank         1000 non-null   int64   
dtypes: category(4), int64(7)
memory usage: 67.0 KB


### Exploratory Data Analysis

Mari lakukan analisis korelasi antar kolom numerik pada dataset kita. Sebelumnya, pastikan bahwa Anda telah memilih kolom-kolom yang bersifat numerik untuk dimasukkan dalam perhitungan korelasi. 

Proses ini akan membantu kita memahami hubungan statistik antara variabel-variabel numerik dalam dataset, sehingga kita dapat mendapatkan wawasan yang lebih mendalam terkait dengan pola atau asosiasi yang mungkin ada di antara mereka.

In [29]:
credit.corr(numeric_only=True)

,Creditability,Duration_of_Credit_monthly,Credit_Amount,Length_of_current_employment,Duration_in_Current_address,Age_years,No_of_Credits_at_this_Bank
Creditability,1.000000,-0.214927,-0.154740,0.116002,-0.002967,0.091272,0.045732
Duration_of_Credit_monthly,-0.214927,1.000000,0.624988,0.057381,0.034067,-0.037550,-0.011284
Credit_Amount,-0.154740,0.624988,1.000000,-0.008376,0.028917,0.032273,0.020785
Length_of_current_employment,0.116002,0.057381,-0.008376,1.000000,0.245081,0.259116,0.125791
Duration_in_Current_address,-0.002967,0.034067,0.028917,0.245081,1.000000,0.265626,0.089625
Age_years,0.091272,-0.037550,0.032273,0.259116,0.265626,1.000000,0.150718
No_of_Credits_at_this_Bank,0.045732,-0.011284,0.020785,0.125791,0.089625,0.150718,1.000000


**Soal 1: Mana dari berikut ini adalah pasangan variabel dengan korelasi tertinggi?**

- [x] A. `Duration_of_Credit_monthly` dan `Credit_Amount`
- [ ] B. `Credit_Amount` dan `Length_of_current_employment`
- [ ] C. `Duration_in_Current_address` dan `No_of_Credits_at_this_Bank`
- [ ] D. `Age_years` dan `No_of_Credits_at_this_Bank`

###  Data Pre-Processing

Terdapat 2 hal yang biasanya dilakukan pada tahapan data pre-processing yaitu **Dummy Variable Encoding** dan juga **Cross Validation**

####  Dummy Variable Encoding 

Variabel yang kita miliki terdapat variabel dengan tipe data category, oleh karena itu kita perlu membuat dummy variabel terlebih dahulu. Untuk algoritma Logistic Regression, karena masih terdapat asumsi multicolinearity, maka yang akan dipakai adalah dummy variable. 
    
Mari lakukan metode tersebut dengan memanfaatkan fungsi berikut ini `pd.get_dummies()` dan mengisinya dengan beberapa parameter antara lain:

- `data`: dataset/dataframe yang ingin diubah menjadi numerikal
- `columns`: list kolom yang akan dilakukan dummy variable encoding
- `drop_first`: apakah ingin drop kolom pertama. Default False. Namun akan kita atur sebagai True agar kolom hasil dummies tidak redundan
- `dtype` = memasukan tipe data yang ingin di-isi

Alasan lain kenapa kita melakukan encoding: komputer tidak mengenali bahasa natural manusia seperti bahasa Inggris, Bahasa Indonesia, dll. Komputer hanya mengenal angka.

In [30]:
# # code here
credit_enc = pd.get_dummies(data = credit, 
                             columns = cat,
                             drop_first = True,
                             dtype ='int')

### Train-Test Splitting

Berikut adalah langkah-langkah yang akan kita lakukan selanjutnya dalam proses ini: 

Pertama, kita akan mempersiapkan data dengan menentukan variabel `X` dan `y`. 

Selanjutnya, kita akan melakukan train-test splitting menggunakan fungsi `train_test_split()`, di mana kita akan mengalokasikan 20% dari data sebagai data uji. Dengan ketentuan sebagai berikut:
- **`random_state=123`** untuk memastikan reproduktivitas. 
- **`stratify = y`** untuk memastikan distribusi kelas yang seimbang dalam set data uji.

In [40]:
from sklearn.model_selection import train_test_split

X = sm.add_constant(credit_enc.drop(columns='Creditability'))
y = credit_enc['Creditability']

X_train, X_test, y_train, y_test = train_test_split(X, # kolom prediktor
                                                   y, # kolom target
                                                   test_size = 0.2, # 80% training and 20% test
                                                   random_state = 123,
                                                   stratify=y)

**Soal 2: Mengapa kita melakukan train-test splitting pada data?**

- [ ] A. Agar model memiliki lebih banyak data untuk training
- [ ] B. Untuk menguji model pada data yang sama dengan yang digunakan untuk pelatihan
- [x] C. Menghindari overfitting dan mengukur kinerja model pada data yang belum pernah dilihat
- [ ] D. Menyederhanakan proses analisis data


### Feature Scaling

Setelah itu, langkah selanjutnya yang perlu diperhatikan adalah memastikan bahwa skala data yang digunakan dalam model logistik regresi seragam. Hal ini penting karena perbedaan skala pada variabel-variabel dapat memengaruhi kinerja model. 

Oleh karena itu, kita akan melakukan proses scaling menggunakan metode `StandarScaler()`. 

Pastikan untuk menggunakan metode `fit_transform` pada data pelatihan (`X_train`). Selanjutnya, kita akan menerapkan transformasi menggunakan metode `transform` pada data uji (`X_test`) 

Dengan cara ini, kita dapat memastikan konsistensi skala data yang digunakan oleh model logistik regresi untuk hasil yang lebih akurat dan stabil.

In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [34]:
# # transform
X_train_scale = scaler.transform(X_train)
X_test_scale = scaler.transform(X_test)

**Soal 3: Mengapa feature scaling di scikit-learn pada data train menggunakan `fit_transform` sedangkan pada data test menggunakan `transform`?**

- [ ] A. Untuk memastikan data test memiliki distribusi yang serupa dengan data train
- [x] B. Agar skala fitur dihitung berdasarkan data train, dan kemudian diterapkan pada data test tanpa mengubah distribusi data test
- [ ] C. Karena transformasi yang berbeda diperlukan untuk data train dan test
- [ ] D. Ini adalah konvensi yang umum digunakan dalam pemrosesan data



### Model Fitting

Selanjutnya, mari kita lanjutkan dengan pembuatan model regresi logistik menggunakan fungsi `LogisticRegression()`. 

Setelah proses tersebut, kita akan melakukan fitting model pada dataset yang telah melalui tahap scaling.

In [35]:
from sklearn.linear_model import LogisticRegression

# # Definisikan untuk Inisialisasi model Logistic Regression.
class_lr = LogisticRegression(random_state=123)

# # Fit (melatih) model dengan data pelatihan 
class_lr.fit(X_train_scale, y_train)

LogisticRegression(random_state=123)

### Model Prediction

Setelah berhasil membuat model, langkah berikutnya adalah melakukan prediksi pada dataset uji (`X_test_scale`) menggunakan fungsi `predict()`. Setelah mendapatkan hasil prediksi, kita akan menampilkan proporsi kelas dari hasil prediksi tersebut menggunakan fungsi `value_counts()`. Sebelumnya, perlu dilakukan transformasi hasil prediksi menjadi objek Series menggunakan fungsi `pd.Series()`. Dengan demikian, kita dapat dengan lebih mudah menganalisis distribusi kelas pada hasil prediksi model regresi logistik tersebut.

In [36]:
# # Prediksi data
y_pred_class = class_lr.predict(X_test_scale)

# # Menampilkan distribusi kelas hasil prediksi
pd.Series(y_pred_class).value_counts()

1    168
0     32
Name: count, dtype: int64

**Soal 4: Bagaimana distribusi data hasil prediksi yang dihasilkan pada model `class_lr` kita untuk setiap kelas?**

- [x] A. Kelas 0 = 168, Kelas 1 = 32 
- [ ] B. Kelas 0 = 182, Kelas 1 = 18
- [ ] C. Kelas 0 = 32, Kelas 1 = 168
- [ ] D. Kelas 0 = 18, Kelas 1 = 182

### Model Evaluation

Selain itu, untuk memastikan keefektifan dan kehandalan model yang telah dibuat, penting bagi kita untuk melakukan evaluasi model. Evaluasi ini dapat melibatkan pengukuran sejumlah metrik performa, seperti akurasi, presisi, dan recall. Untuk melakukan pengukuran ini, kita dapat memanfaatkan modul `metrics` dari pustaka scikit-learn dengan mengimpor `from sklearn import metrics`. Dengan menghitung nilai-nilai akurasi, presisi, dan recall, kita dapat memperoleh pemahaman yang lebih komprehensif tentang sejauh mana model kita dapat mengklasifikasikan data dengan benar dan efisien. Ini merupakan langkah penting dalam mengevaluasi kecanggihan model regresi logistik yang telah kita bangun.

In [37]:
from sklearn import metrics

# # Menghitung akurasi
accuracy_lr = metrics.accuracy_score(y_test, y_pred_class)

# # Menghitung presisi
precision_lr = metrics.precision_score(y_test, y_pred_class)

# # Menghitung recall
recall_lr = metrics.recall_score(y_test, y_pred_class)

In [38]:
print(accuracy_lr)
print(precision_lr)
print(recall_lr)

0.72
0.75
0.9


In [39]:
y.value_counts()

Creditability
1    700
0    300
Name: count, dtype: int64

**Soal 5: Sebagai seorang yang bekerja dalam analisis risiko kredit, metrik evaluasi mana yang paling relevan untuk menilai kemampuan model dalam mendeteksi kredit yang berpotensi bermasalah?**

   - [x] A. Recall
   - [ ] B. Specificity
   - [ ] C. Accuracy
   - [ ] D. Precision
